In [1]:
import pandas as pd
import numpy as np
import json
import glob
from urllib.request import urlopen
import wikipediaapi
import random 
from collections import Counter 

In [2]:
wiki_wiki = wikipediaapi.Wikipedia('ta')
parent_cats_df = pd.read_csv("data/disciplines.csv") 
parent_cats = parent_cats_df['Tamil Title'].tolist()
parent_cats
skip_cats = ["Commons category with local link same as on Wikidata"]
parent_cats

['ஓவியம்',
 'கட்டிடக்கலை',
 'சிற்பக்கலை',
 'ஒளிப்படவியல்',
 'நாடகம்',
 'இசை',
 'நடனம்',
 'திரைப்படம்',
 'மொழி',
 'இலக்கியம்',
 'வரலாறு',
 'சட்டம்',
 'மெய்யியல்',
 'சமயம்',
 'மானிடவியல்',
 'தொல்லியல்',
 'பொருளியல்',
 'அரசியல்',
 'உளவியல்',
 'சமூகவியல்',
 'உயிரியல்',
 'வேதியியல்',
 'புவியியல்',
 'புவி அறிவியல்',
 'வானியல்',
 'இயற்பியல்',
 'கணினியியல்',
 'கணிதம்',
 'புள்ளியியல்',
 'வணிகவியல்',
 'தொழினுட்பம்',
 'நலம்',
 'நூலகவியல்',
 'ஊடகவியல்',
 'கல்வி',
 'போரியல்',
 'சமூகம்',
 'பண்பாடு']

In [3]:
def get_cats(wiki_wiki, page_name):
    page_py = wiki_wiki.page(page_name)
    page_cats = False
    if page_py.exists() == True:
        page_cats = page_py.categories
        page_cats = list(page_cats.keys())
    return page_cats

In [4]:
def find_cat(wiki_wiki, parent_cats, category_name, recursion_count):
    result = "NoCat"
    
    ## base/terminating condition
    if recursion_count > 25:
        return result, recursion_count
    recursion_count = recursion_count + 1
    
    skip_cats = ["Commons category with local link same as on Wikidata", "Commons category with local link different than on Wikidata", "விக்கிப்பீடியா", "மேற்கோள் தேவைப்படும் அனைத்து கட்டுரைகள்", "விக்கிப்பீடியா துப்புரவு", "விக்கிப்பீடியா நிர்வாகம்"]
    cat1 = category_name.replace("பகுப்பு:", "")
    
    ## base/terminating condition
    if cat1 in parent_cats :
        return cat1, recursion_count
    else: 
        page_cats = get_cats(wiki_wiki, category_name)
        
        if page_cats == False:
            return result, recursion_count

        # shuffle so that we randomize the category selection
        random.shuffle(page_cats) 

        # If cat is in the page_cats, return
        for cat in page_cats: 
            cat2 = cat.replace("பகுப்பு:", "")
            
            if cat2 in skip_cats :
                continue
    
            if cat2 in parent_cats :
                result = cat2
                return cat2, recursion_count
            
        # Else continue to search
        for cat in page_cats: 
            cat2 = cat.replace("பகுப்பு:", "")
            
            if cat2 in skip_cats :
                continue
            
            result, recursion_count = find_cat(wiki_wiki, parent_cats, cat, recursion_count)    
                
            ## base/terminating condition                
            if result in parent_cats :
                return result, recursion_count          
            
            # Not sure if we will ever get here
            # Call and break, otherwise, we will add too many things to the stack
            break
    
    return result, recursion_count

In [5]:
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

In [6]:
## Read the artile info into a dataframe
json_files = glob.glob('data/tamil_wiki_data/wiki_*')
df_list = []
for filename in json_files:
    df_list.append(pd.read_json(filename, orient='columns', lines=True))
    
df = pd.concat(df_list, ignore_index=True)    
df = df.sort_values(by=['id'])
df.head(10)

,categories,id,text,title,url
43439,[],3,முதற் பக்கம்\n\nBULLET::::- விக்கிப்பீடியா மொழ...,முதற் பக்கம்,https://ta.wikipedia.org/wiki?curid=3
43440,[கட்டிடக்கலை],12,கட்டிடக்கலை\n\nகட்டிடக்கலை என்பது கட்டிடங்கள் ...,கட்டிடக்கலை,https://ta.wikipedia.org/wiki?curid=12
43441,[கட்டிடக்கலை],13,கட்டிடங்களின் பட்டியல்\n\nபிரபலமான அல்லது குறி...,கட்டிடங்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=13
43442,"[கட்டிடக்கலை, கட்டிடக்கலைஞர்கள்]",16,கட்டடக் கலைஞர்\n\nஒரு கட்டிடக்கலைஞன் அல்லது கட...,கட்டடக் கலைஞர்,https://ta.wikipedia.org/wiki?curid=16
43443,[கட்டிடக்கலைஞர்கள்],17,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்\n...,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=17
43444,"[புவியியல், புவி அறிவியல்]",18,"புவியியல்\n\nபுவியியல் என்பது புவி, அங்குள்ள ந...",புவியியல்,https://ta.wikipedia.org/wiki?curid=18
43445,"[நாடுகள், நாடுகள் தொடர்பான பட்டியல்கள்]",19,உலக நாடுகள் பட்டியல் (அகர வரிசையில்)\n\nஇது உல...,உலக நாடுகள் பட்டியல் (அகர வரிசையில்),https://ta.wikipedia.org/wiki?curid=19
43446,"[நாடுகாண் பயணிகள், இத்தாலியர், 1451 பிறப்புகள்...",21,கிறித்தோபர் கொலம்பசு\n\nகிறித்தோபர் கொலம்பசு (...,கிறித்தோபர் கொலம்பசு,https://ta.wikipedia.org/wiki?curid=21
43447,[இலங்கை புவியியல்],26,"இலங்கையின் புவியியல்\n\nஇலங்கை, இந்தியாவுக்குத...",இலங்கையின் புவியியல்,https://ta.wikipedia.org/wiki?curid=26
43448,"[இந்தியப் புவியியல், ஆசியா, தெற்கு ஆசியா, கண்ட...",32,இந்தியத் துணைக்கண்டம்\n\nஇந்தியத் துணைக் கண்டம...,இந்தியத் துணைக்கண்டம்,https://ta.wikipedia.org/wiki?curid=32


In [7]:
df.shape

(129526, 5)

In [8]:
df_subset = df.iloc[1:10]
df_subset = df_subset.sort_values(by=['id'])
df_subset

,categories,id,text,title,url
43440,[கட்டிடக்கலை],12,கட்டிடக்கலை\n\nகட்டிடக்கலை என்பது கட்டிடங்கள் ...,கட்டிடக்கலை,https://ta.wikipedia.org/wiki?curid=12
43441,[கட்டிடக்கலை],13,கட்டிடங்களின் பட்டியல்\n\nபிரபலமான அல்லது குறி...,கட்டிடங்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=13
43442,"[கட்டிடக்கலை, கட்டிடக்கலைஞர்கள்]",16,கட்டடக் கலைஞர்\n\nஒரு கட்டிடக்கலைஞன் அல்லது கட...,கட்டடக் கலைஞர்,https://ta.wikipedia.org/wiki?curid=16
43443,[கட்டிடக்கலைஞர்கள்],17,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்\n...,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=17
43444,"[புவியியல், புவி அறிவியல்]",18,"புவியியல்\n\nபுவியியல் என்பது புவி, அங்குள்ள ந...",புவியியல்,https://ta.wikipedia.org/wiki?curid=18
43445,"[நாடுகள், நாடுகள் தொடர்பான பட்டியல்கள்]",19,உலக நாடுகள் பட்டியல் (அகர வரிசையில்)\n\nஇது உல...,உலக நாடுகள் பட்டியல் (அகர வரிசையில்),https://ta.wikipedia.org/wiki?curid=19
43446,"[நாடுகாண் பயணிகள், இத்தாலியர், 1451 பிறப்புகள்...",21,கிறித்தோபர் கொலம்பசு\n\nகிறித்தோபர் கொலம்பசு (...,கிறித்தோபர் கொலம்பசு,https://ta.wikipedia.org/wiki?curid=21
43447,[இலங்கை புவியியல்],26,"இலங்கையின் புவியியல்\n\nஇலங்கை, இந்தியாவுக்குத...",இலங்கையின் புவியியல்,https://ta.wikipedia.org/wiki?curid=26
43448,"[இந்தியப் புவியியல், ஆசியா, தெற்கு ஆசியா, கண்ட...",32,இந்தியத் துணைக்கண்டம்\n\nஇந்தியத் துணைக் கண்டம...,இந்தியத் துணைக்கண்டம்,https://ta.wikipedia.org/wiki?curid=32


In [9]:
cat_list = []
recursion_depth_list = []

i = 0;
for index, row in df_subset.iterrows():
    cats = row["categories"]
    title = row["title"]
    i = i + 1
    print("item: " + str(i))
    print("title: " + title)
    
    parent_cat_list = []
    recursion_count_list = []
    for cat in cats:
        category_name = "பகுப்பு:" + cat
        found_cat, recursion_count = find_cat(wiki_wiki, parent_cats, category_name, 0)
        parent_cat_list.append(found_cat)
        recursion_count_list.append(recursion_count)

    if len(cats) == 0:
        parent_cat = "NoCat"
        recursion_depth = 99
    else:
        parent_cat = most_frequent(parent_cat_list)
        recursion_count_index = parent_cat_list.index(parent_cat)
        recursion_depth = recursion_count_list[recursion_count_index]
        
    cat_list.append(parent_cat)
    recursion_depth_list.append(recursion_depth)
    print("category: " + parent_cat)

item: 1
title: கட்டிடக்கலை
category: கட்டிடக்கலை
item: 2
title: கட்டிடங்களின் பட்டியல்
category: கட்டிடக்கலை
item: 3
title: கட்டடக் கலைஞர்
category: கட்டிடக்கலை
item: 4
title: குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்
category: கட்டிடக்கலை
item: 5
title: புவியியல்
category: புவியியல்
item: 6
title: உலக நாடுகள் பட்டியல் (அகர வரிசையில்)
category: புவியியல்
item: 7
title: கிறித்தோபர் கொலம்பசு
category: வரலாறு
item: 8
title: இலங்கையின் புவியியல்
category: புவியியல்
item: 9
title: இந்தியத் துணைக்கண்டம்
category: புவியியல்


In [10]:
df_subset['parent_category'] = cat_list
df_subset['parent_category_recursion_depth'] = recursion_depth_list
df_subset

,categories,id,text,title,url,parent_category,parent_category_recursion_depth
43440,[கட்டிடக்கலை],12,கட்டிடக்கலை\n\nகட்டிடக்கலை என்பது கட்டிடங்கள் ...,கட்டிடக்கலை,https://ta.wikipedia.org/wiki?curid=12,கட்டிடக்கலை,1
43441,[கட்டிடக்கலை],13,கட்டிடங்களின் பட்டியல்\n\nபிரபலமான அல்லது குறி...,கட்டிடங்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=13,கட்டிடக்கலை,1
43442,"[கட்டிடக்கலை, கட்டிடக்கலைஞர்கள்]",16,கட்டடக் கலைஞர்\n\nஒரு கட்டிடக்கலைஞன் அல்லது கட...,கட்டடக் கலைஞர்,https://ta.wikipedia.org/wiki?curid=16,கட்டிடக்கலை,1
43443,[கட்டிடக்கலைஞர்கள்],17,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்\n...,குறிப்பிடத்தக்க கட்டிடக்கலைஞர்களின் பட்டியல்,https://ta.wikipedia.org/wiki?curid=17,கட்டிடக்கலை,1
43444,"[புவியியல், புவி அறிவியல்]",18,"புவியியல்\n\nபுவியியல் என்பது புவி, அங்குள்ள ந...",புவியியல்,https://ta.wikipedia.org/wiki?curid=18,புவியியல்,1
43445,"[நாடுகள், நாடுகள் தொடர்பான பட்டியல்கள்]",19,உலக நாடுகள் பட்டியல் (அகர வரிசையில்)\n\nஇது உல...,உலக நாடுகள் பட்டியல் (அகர வரிசையில்),https://ta.wikipedia.org/wiki?curid=19,புவியியல்,1
43446,"[நாடுகாண் பயணிகள், இத்தாலியர், 1451 பிறப்புகள்...",21,கிறித்தோபர் கொலம்பசு\n\nகிறித்தோபர் கொலம்பசு (...,கிறித்தோபர் கொலம்பசு,https://ta.wikipedia.org/wiki?curid=21,வரலாறு,7
43447,[இலங்கை புவியியல்],26,"இலங்கையின் புவியியல்\n\nஇலங்கை, இந்தியாவுக்குத...",இலங்கையின் புவியியல்,https://ta.wikipedia.org/wiki?curid=26,புவியியல்,2
43448,"[இந்தியப் புவியியல், ஆசியா, தெற்கு ஆசியா, கண்ட...",32,இந்தியத் துணைக்கண்டம்\n\nஇந்தியத் துணைக் கண்டம...,இந்தியத் துணைக்கண்டம்,https://ta.wikipedia.org/wiki?curid=32,புவியியல்,2


In [11]:
df_subset.to_csv("data/ready_data_0_10_example.csv")